# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [10]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
loan_stats = pd.read_csv(file_path)

# Preview the data
loan_stats.head()
loan_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 86 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   68817 non-null  float64
 1   int_rate                    68817 non-null  float64
 2   installment                 68817 non-null  float64
 3   home_ownership              68817 non-null  object 
 4   annual_inc                  68817 non-null  float64
 5   verification_status         68817 non-null  object 
 6   issue_d                     68817 non-null  object 
 7   loan_status                 68817 non-null  object 
 8   pymnt_plan                  68817 non-null  object 
 9   dti                         68817 non-null  float64
 10  delinq_2yrs                 68817 non-null  float64
 11  inq_last_6mths              68817 non-null  float64
 12  open_acc                    68817 non-null  float64
 13  pub_rec                     688

## Split the Data into Training and Testing

In [ ]:
# # Create our features
# X = loan_stats.drop(columns='loan_status')
# X = pd.get_dummies(X, columns=['homeowner'])

# # Create our target
# y = loan_stats['loan_status'].to_frame()
# encoder = LabelEncoder()
# encoder.fit(y['loan_status'])
# y_encoded = encoder.transform(y['loan_status'])
# y['loan_status'] = y_encoded

In [ ]:
X.describe()

In [ ]:
# Check the balance of our target values


In [ ]:
# Split the X and y into X_train, X_test, y_train, y_test


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [ ]:
# Create the StandardScaler instance


In [ ]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset


In [ ]:
# Scale the training and testing data


## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [ ]:
# Resample the training data with the BalancedRandomForestClassifier


In [ ]:
# Calculated the balanced accuracy score


In [ ]:
# Display the confusion matrix


In [ ]:
# Print the imbalanced classification report


In [ ]:
# List the features sorted in descending order by feature importance


### Easy Ensemble Classifier

In [ ]:
# Train the Classifier


In [ ]:
# Calculated the balanced accuracy score


In [ ]:
# Display the confusion matrix


In [ ]:
# Print the imbalanced classification report


### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.